# データセット作成

In [1]:
import pandas as pd

### csv読み込み

In [2]:
csv_path_fine_train = ("./datasets_csv/finetuning/train")
csv_path_fine_test = ("./datasets_csv/finetuning/test")
csv_path_pred_labeling = ("./datasets_csv/pred_labeling")

In [3]:
# ニュース記事

#df_finetuning_news = pd.read_csv(csv_path_fine + '/news/news_dataset.csv')
df_train_news = pd.read_csv(csv_path_pred_labeling + '/news/news_dataset.csv')

#print(df_finetuning_news)
print(df_train_news)


                   date                                              title  \
0   2020-07-25 11:08:00  エンタメ業界は苦しい。でも…　「新型コロナ以前の世界が戻ってくることはない」エイベックスが見...   
1   2020-08-03 06:10:00                    オンラインライブ配信はウィズコロナ時代の救世主になれるのか\n   
2   2020-08-11 19:35:00  米Twitchでの音楽使用料に関してミュージシャンから親会社アマゾンのジェフ・ベゾスへ公開書簡\n   
3   2020-09-28 18:20:00  クラシック音楽界の概念を超えた大型オンラインフェスのレポート到着　アーティストによるライブ＆...   
4   2020-10-28 08:00:00    「鬼滅の刃」人気は株式市場にも影響　たまごっち登場、グッズ関連にぎわう…“鬼滅銘柄”を物色\n   
5   2020-10-29 08:10:00                  ソニー好決算「上方修正」。その強みはこの1枚のスライドでわかる\n   
6   2020-11-05 16:24:00                                  エイベックス、初の希望退職募集\n   
7   2020-11-06 12:46:00               エイベックスが初の希望退職募集　コロナ禍でライブ・舞台関連事業に打撃\n   
8   2020-11-07 17:01:00                 嵐や白石麻衣は大成功、音楽業界の「勝ち組」がごく一部だという現実\n   
9   2020-11-17 07:00:00           電子チケット販売に自由を。ZAIKO COOが見据える「D2F」という勝ち筋\n   
10  2020-12-10 19:15:00                      音楽業界の未来、実はストリーミングではなくSNSが重要\n   
11  2020-12-21 18:42:00     浜崎あゆみ、クリスマスと大みそかライブの有観客断念を発表　無観客生配信へ

In [4]:
# ニュースコメント

import os

csv_folder = [csv_path_fine_train , csv_path_fine_test, csv_path_pred_labeling]

file_count = 0
for p in csv_folder:
    file_count += sum((len(f) for _, _, f in os.walk(p + '/comments'))) - 1
#print(file_count)

for j, p in enumerate(csv_folder):
    cols = ['text', 'reply', 'good', 'bad']
    df_temp = pd.DataFrame(columns=cols)
    
    for i in range(file_count):
        n_file = str(i+1).zfill(3)
        file_name = "comment_dataset_" + n_file + ".csv"
        file_path = (p + "/comments/" + file_name)
        if not os.path.isfile(file_path):
            continue
        #print(str(i+1).zfill(3))#あとで消す
        df_cmt = pd.read_csv(file_path, index_col=0)
        #df_temp = pd.concat([df_temp, df_cmt], ignore_index=True)
              
        #代入
        if j <= 1:
            df_temp = pd.concat([df_temp, df_cmt], ignore_index=True)
        else:
            df_cmt.columns = ["feature", "reply", "good", "bad"]
            df_cmt[["feature", "good", "bad"]].to_csv("./datasets/pred_labeling/features_" + n_file + ".csv", index=False)
            #df_pred_comments = df_temp.copy()
        
    #代入
    if j == 0:
        df_fine_train_comments = df_temp.copy()
    elif j == 1:
        df_fine_test_comments = df_temp.copy()
        
print("finetuning train:\n", df_fine_train_comments)
print("finetuning test:\n", df_fine_test_comments)
#print("pred labeling:\n", df_pred_comments)

finetuning train:
                                                  text reply good bad
0   ハロプロの事務所も昨年末に移転しましたよね\n都内から都内だけど、複数の拠点を1拠点にまとめ...     1  278  53
1   日本の芸能プロダクションも\nアメリカのエージェントシステムに\n移行していますね！\nタレ...     0  235  45
2   エイベックスの件はともかくこれから都心の地価は下がるのではないか。これだけリモートワークが推...     3  120  17
3   その昔大学が都心から郊外や県外に脱出しました（中央大学や筑波大学など）。その結果として、人気...     1  107  20
4   河口湖は東京都心からやや遠いけれど、都心回帰で空きが増えた八王子の大学のキャンパスなんか移転...     2   70   5
..                                                ...   ...  ...  ..
56                            本当ただの妄想コントでしかない記事だね\n\n     0   17   0
57                                   浅草をバカにしてるのか？\n\n     0   18   1
58                 日本のショービジネスの人間はアマチュア。\n辞めた方が良い。\n\n     0   11   3
59                                     足立区でよいのでは？\n\n     1   10   6
60                                八王子あたりでよくない？(笑)\n\n     0   21   8

[61 rows x 4 columns]
finetuning test:
                                                 text reply good bad
0  役員報酬がバカ高く上層部ばかりが得する仕組みをやってる会社はどこも衰退していくよ。

In [5]:
#ラベル
import os

csv_folder = [csv_path_fine_train , csv_path_fine_test]

#file_count = 0
#for p in csv_folder:
#    file_count += sum((len(f) for _, _, f in os.walk(p + '/comments'))) - 1
#print(file_count)

for j, p in enumerate(csv_folder):
    cols = ['label']
    df_temp = pd.DataFrame(columns=cols)

    for i in range(file_count):
        n_file = str(i+1).zfill(3)
        file_name = "comment_labels_" + n_file + ".csv"
        file_path = (p + "/labels/" + file_name)
        if not os.path.isfile(file_path):
            continue
        #print(str(i+1).zfill(3))#あとで消す
        df_labels = pd.read_csv(file_path)#, index_col=0
        # concat
        #print(df_labels)
        df_temp = pd.concat([df_temp, df_labels], ignore_index=True)

    #代入
    if j == 0:
        df_fine_train_labels = df_temp.copy()
    elif j == 1:
        df_fine_test_labels = df_temp.copy()
        
print("labels train:\n", df_fine_train_labels)
print("labels test:\n", df_fine_test_labels)

labels train:
        label
0   positive
1   positive
2   positive
3   negative
4   positive
..       ...
56  negative
57  negative
58  negative
59  negative
60  negative

[61 rows x 1 columns]
labels test:
       label
0  negative
1  negative
2  negative
3  negative
4  positive
5  negative
6  negative
7  negative
8  negative
9  positive


In [6]:
# データセットの作成と保存

datasets_folder =  ("./datasets/finetuning")


#print("finetuning train:\n", df_fine_train_comments)
#print("finetuning test:\n", df_fine_test_comments)
#print("pred labeling:\n", df_pred_comments)

df_fine_train_comments.columns = ["feature", "reply", "good", "bad"]
df_fine_train_comments["feature"].to_csv(datasets_folder + "/train/features.csv", index=False)

df_fine_test_comments.columns = ["feature", "reply", "good", "bad"]
df_fine_test_comments["feature"].to_csv(datasets_folder + "/test/features.csv", index=False)

#print("labels train:\n", df_fine_train_labels)
#print("labels test:\n", df_fine_test_labels)

df_fine_train_labels["label"].to_csv(datasets_folder + "/train/labels.csv", index=False)
df_fine_test_labels["label"].to_csv(datasets_folder + "/test/labels.csv", index=False)

# 上に持っていく
#df_pred_comments.columns = ["feature", "reply", "good", "bad"]
#df_pred_comments["feature"].to_csv("./datasets/pred_labeling/features.csv", index=False)

    

In [7]:
#表示用
#news_file_path = (p + "/labels/" + file_name)
#df_cmt = pd.read_csv(news_file_path, index_col=0)
#df_temp = pd.concat([df_temp, df_cmt], ignore_index=True)
#
#dsp_df_fine_train_comments = df_fine_train_comments["feature"]
#dsp_df_fine_train_labels = df_fine_train_labels["label"]
#dsp_df_pred_comments = df_pred_comments["feature"]

In [8]:
#ニュースの例
#dsp_df_pred_comments[0]

In [9]:
#コメントの例（上のニュースとは無関係）
#dsp_df_fine_train_comments[0]

In [10]:
#ラベルの例（上のニュースとは無関係）
#dsp_df_fine_train_labels[0]

In [11]:
# データセットの表示(一旦保留)

#df_finetuning_news["feature"] = df_finetuning_news["title"] +  df_finetuning_news["text"]
#df_finetuning_news = df_finetuning_news["feature"].copy()
#print(df_finetuning_news)


#df_train_news["feature"] = df_train_news["title"] +  df_train_news["text"]
#df_train_news = df_train_news["feature"].copy()
#print(df_train_news)
#
#df_finetuning_comments.columns = ["feature", "reply", "good", "bad"]
#df_finetuning_comments = df_finetuning_comments["feature"].copy()
#print(df_finetuning_comments)
#
#df_train_comments.columns = ["feature", "reply", "good", "bad"]
#df_train_comments = df_train_comments["feature"].copy()
#print(df_train_comments)
#
#df_finetuning_labels = df_finetuning_labels["label"].copy()
#print(df_finetuning_labels)
#
# return (df_finetuning_news["feature"], 
#                df_train_news["feature"], 
#                df_finetuning_comments["text"], 
#                df_train_comments["text"],
#                df_finetuning_labels["label"])

In [12]:
#df_finetuning_news[0]

In [13]:
#df_finetuning_comments[0]

In [14]:
#df_finetuning_labels[0]

In [15]:
#df_finetuning_comments[1]

In [16]:
#df_finetuning_labels[1]

In [17]:
#df_finetuning_comments[2]

In [18]:
#df_finetuning_labels[2]

In [19]:
#!pip install bert-tensorflow
#!pip install keras-bert

In [20]:
#!pip install sentencepiece

https://tech-blog.cloud-config.jp/2020-02-06-category-classification-using-bert/

In [21]:
#!git clone --recurse-submodules https://github.com/yoheikikuta/bert-japanese

## 設定ファイル

In [22]:
import pandas as pd
import sentencepiece as spm

# 学習データX
# feature.csvは上記で用意したファイルのパスを指定してください
train_features_df = pd.read_csv('./datasets/finetuning/train/features.csv')
print("train_features_df:\n", train_features_df)


# テキストの最初に[CLS]、最後に'[SEP]をつけて単語数を数える関数
def _get_indice(feature):
    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))# sentence piece
    tokens.append('[SEP]')
    print("テキストのデータ :\n",tokens)
    number = len(tokens)
    return number

# sentence pieceでは、与えた文書の中で高い頻度で現れるフレーズは、
# 多少長くても一つの単位として認識します。
# Mecabでは対応する辞書を使って文章を分割します。辞書にはneologdとかがよく使われます。
#これでも上手くいくことも多いですが、語彙数が大きくなってしまうことや、
# 分割の仕方が分割したいデータセットに適していないこともあり、問題点
sp = spm.SentencePieceProcessor()

# ダウンロードした事前学習モデルのパスを指定してください
sp.Load('./downloads/bert-wiki-ja/wiki-ja.model')

numbers = []
for feature in train_features_df['feature']:
    features_number = _get_indice(feature)
    numbers.append(features_number)

print(numbers)

# 最大トークン数
max_token_num = max(numbers)
print("max_token_number: " + str(max_token_num))

train_features_df:
                                               feature
0   ハロプロの事務所も昨年末に移転しましたよね\n都内から都内だけど、複数の拠点を1拠点にまとめ...
1   日本の芸能プロダクションも\nアメリカのエージェントシステムに\n移行していますね！\nタレ...
2   エイベックスの件はともかくこれから都心の地価は下がるのではないか。これだけリモートワークが推...
3   その昔大学が都心から郊外や県外に脱出しました（中央大学や筑波大学など）。その結果として、人気...
4   河口湖は東京都心からやや遠いけれど、都心回帰で空きが増えた八王子の大学のキャンパスなんか移転...
..                                                ...
56                            本当ただの妄想コントでしかない記事だね\n\n
57                                   浅草をバカにしてるのか？\n\n
58                 日本のショービジネスの人間はアマチュア。\n辞めた方が良い。\n\n
59                                     足立区でよいのでは？\n\n
60                                八王子あたりでよくない？(笑)\n\n

[61 rows x 1 columns]
テキストのデータ :
 ['[CLS]', '▁', 'ハロ', 'プロ', 'の', '事務所', 'も', '昨', '年末に', '移転', 'しま', 'した', 'よ', 'ね', '▁', '都内', 'から', '都内', 'だけ', 'ど', '、', '複数の', '拠点', 'を', '1', '拠点', 'に', 'まとめた', 'みたい', '▁', '経営', '立て直し', 'する', '上で', '、', '固定', '費', 'の中でも', 'まず', '着手', 'するのは', '家', '賃', 'です', 'から', 'ね', '。', '▁', 'し', 'んど', 'い

## 学習

## BERTの設定ファイル、モデルのロード  
学習回数と事前に調べていた最大トークン数、ファイルパスを自分用に書き換えてください。以下に書き換える箇所を示します。  

- config_path：設定ファイルのパス  
- checkpoint_path：事前学習モデルのファイルパス  
    - 拡張子まで書かないでください  
- SEQ_LEN：最大トークン数  
- EPOCH：学習回数  

In [23]:
import sys
from keras_bert import load_trained_model_from_checkpoint

#sys.pathに追加（必要なのか調査が必要）
sys.path.append('modules')

#import pprint
#pprint.pprint(sys.path)

# BERTのロード
config_path = './downloads/bert-wiki-ja_config/bert_finetuning_config_v1.json'
# 拡張子まで記載しない（.ckptファイルで保存されている）
checkpoint_path = './downloads//bert-wiki-ja/model.ckpt-1400000'

# 最大のトークン数
SEQ_LEN = max_token_num#上の処理の出力
BATCH_SIZE = 16
BERT_DIM = 768
LR = 1e-4
# 学習回数
EPOCH = 20

# 学習ずみモデルでモデル構築
bert = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True,  trainable=True, seq_len=SEQ_LEN)
bert.summary()

# この後に追加する（転移学習）
# 分類問題用にモデルの再構築

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 206)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 206)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 206, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 206, 768)     1536        Input-Segment[0][0]              
______________________________________________________________________________________________

## 学習データのロード関数
こちらの関数でもモデルの読み込みを行うので、各自ファイルパスの変更をお願いします。

- sp.load("ファイルパス")

## 文章のベクトル化
_get_indice関数では、SentencePieceとwikipediaモデルを使用し文章のベクトル化を行っています

## 学習データ読込
_load_labeldata関数は学習データを読込、_get_indice関数を用いて特徴量を抽出しています。




In [24]:
from keras.utils import np_utils

In [25]:
from keras import utils
import numpy as np

# ここでもsentence piece
sp = spm.SentencePieceProcessor()
sp.Load('./downloads/bert-wiki-ja/wiki-ja.model')

# 上に同じ名前の関数があるので注意
# 最大単語数分のID化された文を返す関数
# maxlenがなくてエラーになるので勝手に追加（maxlenは最大単語数か？）
def _get_indice(feature, maxlen):
#def _get_indice(feature):
    # インデックス ０で埋める
    indices = np.zeros((maxlen), dtype = np.int32)
    # 最初に[CLS]、最後に'[SEP]をつけてトークン作る
    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))# sentence piece
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        # 最大単語数までトークンの単語をindicesに入れていく
        if t >= maxlen:
            break
        try:
            indices[t] = sp.piece_to_id(token)# id化してくれる？
        except:
            logging.warn(f'{token} is unknown.')# コメントしてくれる
            indices[t] = sp.piece_to_id('<unk>')# id化してくれる？unknown
    
    # 最大単語数分のID化された文を返す
    return indices


#勝手に追加 maxlen=103  
# 引数のパスは直接書けばいらないかも
def _load_labeldata(train_dir, test_dir, maxlen):
    # pandasでcsvの学習データとテストデータを読み込む
    train_features_df = pd.read_csv(f'{train_dir}/features.csv')
    train_labels_df = pd.read_csv(f'{train_dir}/labels.csv')
    test_features_df = pd.read_csv(f'{test_dir}/features.csv')
    test_labels_df = pd.read_csv(f'{test_dir}/labels.csv')
    
    ##### ラベル側の処理 #####
    
    # ラベルのユニーク値を取り出す（ラベル数）（インデックスとラベル別別に保管）
    # ネガポジなら　ポジティブ, ネガティブ と　０、１　を入れてしまえばいいと思われる
    #{'スポーツ': 0, '携帯電話': 1},
    label2index = {k: i for i, k in enumerate(train_labels_df['label'].unique())}
    #{0: 'スポーツ', 1: '携帯電話'}
    index2label = {i: k for i, k in enumerate(train_labels_df['label'].unique())}
    #　クラス数（何種類に分類するか）ネガポジなら２
    class_count = len(label2index)
    
    # Numpyユーティリティ to_categorical(y, nb_classes=None)
    # クラスベクトル（0からnb_classesまでの整数）を categorical_crossentropyとともに用いるためのバイナリのクラス行列に変換します．
    # y: 行列に変換するクラスベクトル, nb_classes: 総クラス数
    # ↓trainのラベルを文字からインデックスを使用して変換
    train_labels = utils.np_utils.to_categorical([label2index[label] for label in train_labels_df['label']], num_classes=class_count)
    #　testのインデックスをまず作る
    test_label_indices = [label2index[label] for label in test_labels_df['label']]
    # ↓testのラベルを文字からインデックスを使用して変換
    test_labels = utils.np_utils.to_categorical(test_label_indices, num_classes=class_count)

    ##### 特徴量側の処理 #####
    
    train_features = []
    test_features = []
    for feature in train_features_df['feature']:
        # 上で作った関数 _get_indice  を使ってID化
        train_features.append(_get_indice(feature, maxlen))
    # shape(len(train_features), maxlen)のゼロの行列作成
    train_segments = np.zeros((len(train_features), maxlen), dtype = np.float32)

    for feature in test_features_df['feature']:
        # 上で作った関数 _get_indice  を使ってID化
        test_features.append(_get_indice(feature, maxlen))
    # shape(len(test_features), maxlen)のゼロの行列作成
    test_segments = np.zeros((len(test_features), maxlen), dtype = np.float32)

    print(f'Trainデータ数: {len(train_features_df)}, Testデータ数: {len(test_features_df)}, ラベル数: {class_count}')

    return {
        'class_count': class_count,
        'label2index': label2index,
        'index2label': index2label,
        'train_labels': train_labels,
        'test_labels': test_labels,
        'test_label_indices': test_label_indices,
        'train_features': np.array(train_features),
        'train_segments': np.array(train_segments),
        'test_features': np.array(test_features),
        'test_segments': np.array(test_segments),
        'input_len': maxlen
    }

In [26]:
_get_indice("制作", maxlen=SEQ_LEN)

array([  4,   9, 828,   5,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [27]:
_load_labeldata('./datasets/finetuning/train', './datasets/finetuning/test', maxlen=SEQ_LEN)

Trainデータ数: 61, Testデータ数: 10, ラベル数: 2


{'class_count': 2,
 'label2index': {'positive': 0, 'negative': 1},
 'index2label': {0: 'positive', 1: 'negative'},
 'train_labels': array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
     

## モデル作成関数

In [28]:
# おそらく、この関数を作った理由は複数分類モデルを自由に作れるようにしたかったからだ。
# 単にネガポジにするなら関数にしないで直接書けばいい。

from keras.layers import Dense, Dropout, LSTM, Bidirectional, Flatten, GlobalMaxPooling1D
from keras_bert.layers import MaskedGlobalMaxPool1D
from keras import Input, Model

# nadam を選べば使わなくてもいい
# https://github.com/CyberZHG/keras-bert
from keras_bert import AdamWarmup, calc_train_steps

def _create_model(input_shape, class_count):
    # AdamWarmupをオプティマイザーとして使用するために必要な情報を得る関数
    # nadam を選べば使わなくてもいい
    decay_steps, warmup_steps = calc_train_steps(
        input_shape[0],
        batch_size=BATCH_SIZE,
        epochs=EPOCH,
    )

    # 学習済みモデル 「bert」 の最終出力層のoutputを取り出す
    bert_last = bert.get_layer(name='NSP-Dense').output
    x1 = bert_last
    # 最終出力層のoutputを新規作成した全結合層に入れる
    output_tensor = Dense(class_count, activation='softmax')(x1)
    
    # Trainableの場合は、Input Masked Layerが3番目の入力なりますが、
    # FineTuning時には必要無いので1, 2番目の入力だけ使用します。
    # Trainableでなければkeras-bertのModel.inputそのままで問題ありません。
    model = Model([bert.input[0], bert.input[1]], output_tensor)
    model.compile(loss='categorical_crossentropy',
                  optimizer=AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
                  #optimizer='nadam',
                  metrics=['mae', 'mse', 'acc'])

    return model

## 学習データのロードとモデルの準備
事前準備で作成した学習用データと学習後のモデル名および出力先を指定してください。

- trains_dir,tests_dir：学習用データのパス
- model_filename：学習後のモデル名、出力先のパス



In [29]:
#import keras.utils

In [30]:
#from keras.utils import np_utils

In [31]:
#from keras import Input, Model, utils
#import numpy as np

In [32]:
# データロードとモデルの準備
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

trains_dir = './datasets/finetuning/train'
tests_dir = './datasets/finetuning/test'

#上で作った関数
data = _load_labeldata(trains_dir, tests_dir, SEQ_LEN)

# モデルの読み込み
model_filename = './downloads/models/knbc_finetuning.model'

# 上で作った関数（関数を使わずに直接書くこともできる）
# data['train_features'].shape　は　文の数×最大単語数　＝　特徴量Xのインプットshape
# data['class_count']　は　クラスの数
model = _create_model(data['train_features'].shape, data['class_count'])

model.summary()

Trainデータ数: 61, Testデータ数: 10, ラベル数: 2
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 206)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 206)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 206, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 206, 768)     1536        Input-Segment[0][0]              
_______________________________________________________

__________________________________________________________________________________________________
Encoder-11-FeedForward (FeedFor (None, 206, 768)     4722432     Encoder-11-MultiHeadSelfAttention
__________________________________________________________________________________________________
Encoder-11-FeedForward-Dropout  (None, 206, 768)     0           Encoder-11-FeedForward[0][0]     
__________________________________________________________________________________________________
Encoder-11-FeedForward-Add (Add (None, 206, 768)     0           Encoder-11-MultiHeadSelfAttention
                                                                 Encoder-11-FeedForward-Dropout[0]
__________________________________________________________________________________________________
Encoder-11-FeedForward-Norm (La (None, 206, 768)     1536        Encoder-11-FeedForward-Add[0][0] 
__________________________________________________________________________________________________
Encoder-12

## 学習の実行
いよいよ学習の実行です。以下のプログラムを実行した際に画像のような出力が出ると思います。（tensorflowのバージョンでWarningが出ますが問題ありません）あとはお茶でも飲みながら学習経過を観察してみましょう！

ファインチューニング？

In [33]:
history = model.fit([data['train_features'], data['train_segments']],
          data['train_labels'],
          epochs = 50,#3
          #epochs = EPOCH,
          batch_size = BATCH_SIZE,
          validation_data=([data['test_features'], data['test_segments']], data['test_labels']),
          shuffle=False,
          verbose = 1,
          callbacks = [
              ModelCheckpoint(monitor='val_acc', mode='max', filepath=model_filename, save_best_only=True)
          ])


Epoch 1/3
4/4 [==============================] - 350s 67s/step - loss: 0.9059 - mae: 0.5712 - mse: 0.3489 - acc: 0.3249 - val_loss: 0.7728 - val_mae: 0.5100 - val_mse: 0.2838 - val_acc: 0.6000
INFO:tensorflow:Assets written to: ./downloads/models/knbc_finetuning.model/assets
Epoch 2/3
4/4 [==============================] - 318s 80s/step - loss: 0.5158 - mae: 0.3761 - mse: 0.1704 - acc: 0.7960 - val_loss: 1.0456 - val_mae: 0.5758 - val_mse: 0.3797 - val_acc: 0.5000
Epoch 3/3
4/4 [==============================] - 289s 72s/step - loss: 0.2538 - mae: 0.1934 - mse: 0.0709 - acc: 0.8898 - val_loss: 0.9273 - val_mae: 0.4750 - val_mse: 0.3043 - val_acc: 0.6000


## 評価および予測
### 評価の出力（不要）
#### 学習経過
学習回数毎の精度を算出し、学習経過を見ることが出来ます。  
※ 学習を行った後に同じnotebookで実行してください。




In [34]:
df = pd.DataFrame(history.history)
display(df)

,loss,mae,mse,acc,val_loss,val_mae,val_mse,val_acc
0,0.908873,0.566745,0.349256,0.327869,0.772762,0.509995,0.283797,0.6
1,0.491979,0.352868,0.162844,0.786885,1.045626,0.575786,0.379701,0.5
2,0.230740,0.179182,0.062932,0.901639,0.927265,0.474997,0.304341,0.6


## モデル評価（不要）
テストデータを用いて学習結果を算出出来ます。ここで用いられている指標については以下のURLを参照してください。
http://tkdmah.hatenablog.com/entry/2014/02/22/193008




In [35]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model
from keras_bert import get_custom_objects

model = load_model(model_filename, custom_objects=get_custom_objects())

predicted_test_labels = model.predict([data['test_features'], data['test_segments']]).argmax(axis=1)
numeric_test_labels = np.array(data['test_labels']).argmax(axis=1)

report = classification_report(
        numeric_test_labels, predicted_test_labels, target_names=['スポーツ', '携帯電話'], output_dict=True)
# , '京都', 'スポーツ'
display(pd.DataFrame(report).T)

,precision,recall,f1-score,support
スポーツ,0.250000,0.5000,0.333333,2.0
携帯電話,0.833333,0.6250,0.714286,8.0
accuracy,0.600000,0.6000,0.600000,0.6
macro avg,0.541667,0.5625,0.523810,10.0
weighted avg,0.716667,0.6000,0.638095,10.0


## 推論時にAttention Weightを出力するようにモデルをロードする

In [36]:
#from keras.models import load_model

# custom_objects 
# custom_objects引数を使ってロード機構にそのカスタムレイヤーなどを渡すことができます
#model = load_model(model_filename, custom_objects=SeqSelfAttention.get_custom_objects())

#ModelクラスAPI   model = Model(inputs=a, outputs=b)  model = Model(inputs=[a1, a2], outputs=[b1, b2, b3])
# model.get_layer('attention').output 2つ目のアウトプットを指定
#model = Model(inputs=model.input, outputs=[model.output, model.get_layer('attention').output])
#model.summary()

## 予測
いよいよ学習したモデルを用いた予測です。流れとしては入力された文章の特徴量を抽出し、モデルに入力するだけの簡単なお仕事です！ただ注意点もあります。それについては一度下記のソースを実行した後に解説します。

In [37]:
tests_features_df = pd.read_csv('./datasets/pred_labeling/features_001.csv')
tests_features_df.loc[0]['feature']

'イギリスから戻って来た\n布袋寅泰氏はどうするのかね\n緊急事態宣言に\n劇場とかは含まれないみたいだけど\n全国からファンを\n武道館に来させるのかな\n中止もしくは\n武道館に来なくても払い戻しします\n位の措置は取った方がいいと思う\n\n'

In [38]:
import sys
import pandas as pd
import sentencepiece as spm
import logging
import numpy as np

from keras import utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import get_custom_objects
from sklearn.metrics import classification_report, confusion_matrix


#sys.pathに追加（必要なのか調査が必要）
sys.path.append('modules')

# SentencePieceProccerモデルの読込
spp = spm.SentencePieceProcessor()
spp.Load('./downloads/bert-wiki-ja/wiki-ja.model')

# BERTの学習したモデルの読込（ダウンロードした？勝手に保存される？）
model_filename = './downloads/models/knbc_finetuning.model'
model = load_model(model_filename, custom_objects=get_custom_objects())
#model = load_model(model_filename, custom_objects=SeqSelfAttention.get_custom_objects())
model = Model(inputs=model.input, outputs=[model.output, model.get_layer('Encoder-12-MultiHeadSelfAttention').output])
# ↑ここでmodel = Model(inputs=a, outputs=b) としてAttentionも出すようにする。


# 上のと同じのを入れると思われるため、消していいかも(ファイルを分けるなら必要)
#SEQ_LEN = 103#206
maxlen = SEQ_LEN
#
# 上にあったのと同じ？ → predictように変更
def _get_indice_pred(feature, maxlen):
    indices = np.zeros((maxlen), dtype=np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(spp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = spp.piece_to_id(token)
        except:
            logging.warn('unknown')
            indices[t] = spp.piece_to_id('<unk>')
            
    return indices, tokens

tests_features_df = pd.read_csv('./datasets/pred_labeling/features_001.csv')
feature = tests_features_df.loc[0]['feature']
#feature = "昨日は携帯電話を買いに行った。"
#feature = "昨日はスポーツしに行った。"

test_features = []
indices, tokens = _get_indice_pred(feature, maxlen)
test_features.append(indices)

#勝手に追加
test_features = np.array(test_features)

test_segments = np.zeros(
    (len(test_features), maxlen), dtype=np.float32)


# model = Modelを使えば推定　predict[0][0]　２次元のリストで返せる。
predicted_test_labels = model.predict([test_features, test_segments])#.argmax(axis=1)
#predict = model.predict(test_features)


print("完了")

完了


In [39]:
print(tokens)

['[CLS]', '▁イギリス', 'から', '戻って', '来た', '▁', '布', '袋', '寅', '泰', '氏は', 'どう', 'する', 'の', 'かね', '▁', '緊急', '事態', '宣言', 'に', '▁', '劇場', 'とか', 'は', '含まれ', 'ない', 'みたい', 'だけ', 'ど', '▁', '全国', 'から', 'ファン', 'を', '▁', '武道', '館', 'に', '来', 'させる', 'の', 'かな', '▁', '中止', 'もしくは', '▁', '武道', '館', 'に', '来', 'なくても', '払い', '戻し', 'します', '▁', '位の', '措置', 'は', '取った', '方が', 'いい', 'と思う', '[SEP]']


In [40]:

print("shape: ", len(predicted_test_labels))
#print("predicted_test_labels: ", predicted_test_labels)
print("predict: ", predicted_test_labels[0])
print("predict: ", predicted_test_labels[0].argmax(axis=1))
print("attention:", predicted_test_labels[1].shape)
print("attention:\n", predicted_test_labels[1])
print("attention:\n", predicted_test_labels[1].argmax(axis=1))#axis=1
print("attention:\n", predicted_test_labels[1].argmax(axis=1).shape)#axis=1

shape:  2
predict:  [[0.42134732 0.5786527 ]]
predict:  [1]
attention: (1, 206, 768)
attention:
 [[[ 0.17543392  0.04414396 -0.43319073 ... -0.16692013 -0.30394632
    0.08585015]
  [ 0.39355442  0.23487765 -0.36567208 ...  0.17807189 -0.12828949
   -0.03393506]
  [ 0.02897491  0.25257352 -0.21223141 ... -0.20154503  0.13764143
   -0.15470809]
  ...
  [ 0.2572218   0.03011176 -0.29198945 ... -0.1734258  -0.29087633
    0.06942913]
  [ 0.12754028  0.11264317 -0.20586725 ... -0.06985087 -0.2634833
    0.05787942]
  [ 0.18480273  0.10476964 -0.2631198  ... -0.01969994 -0.34040308
    0.10731167]]]
attention:
 [[ 42  31   4  44   7  47   7  57  52  17  44   4  48  41  22  49  24   7
   22  45  27  59  14  47  22   7  41  25  48   0  21  40  59  59   4  26
   49  31   3  37  38  27  14  44  25   4  49  41  52  38  60  33  48  41
   56  19  27  46  15  41  41  19  59   6  44  60   4  47   4  19  31  25
    4  31  25  49   7  24  22  41   7   7  31  59  33  41  44  47  38  57
   12  31   1  3

_____________
Encoder-12-MultiHeadSelfAttenti (None, 206, 768)     0           Encoder-11-FeedForward-Norm[0][0]
                                                                 Encoder-12-MultiHeadSelfAttention
__________________________________________________________________________________________________
Encoder-12-MultiHeadSelfAttenti (None, 206, 768)     1536        Encoder-12-MultiHeadSelfAttention

In [41]:
# 入力シーケンスはpad_sequenceにより、以下の様に0でpre paddingしています。
# [0 0 0 0 x1(300) x2(300) x3(300)]
# Attention Weightは入力シーケンスに対応して計算されるため、
# 入力シーケンスのpadding分シフトします。
weights = [w.max() for w in predicted_test_labels[1][0][-len(tokens):]]
df = pd.DataFrame([tokens, weights], index=['token', 'weight']).T

mean = np.asarray(weights).mean()#np.asarray　参照コピー

df['rank'] = df['weight'].rank(ascending=False)#ランキング
# wから平均を引いた値が0より大きいものだけ（偏差）
df['normalized'] = df['weight'].apply(lambda w: max(w - mean, 0))#行全体や列全体に対して、同じ操作
df['weight'] = df['weight'].astype('float32')
df['attention'] = df['normalized'] > 0
# df.style.background_gradient で色つけ
df = df.style.background_gradient(cmap='Blues', subset=['normalized'])

display(df)

,token,weight,rank,normalized,attention
0,[CLS],0.456269,54.000000,0.000000,False
1,▁イギリス,0.459340,53.000000,0.000000,False
2,から,0.432476,56.000000,0.000000,False
3,戻って,0.666447,22.000000,0.077755,True
4,来た,0.713633,9.000000,0.124941,True
5,▁,0.643299,27.000000,0.054607,True
6,布,0.569297,38.000000,0.000000,False
7,袋,0.322138,63.000000,0.000000,False
8,寅,0.340100,61.000000,0.000000,False
9,泰,0.455261,55.000000,0.000000,False


## 予測 作り直し

In [42]:
import sys
import pandas as pd
import sentencepiece as spm
import logging
import numpy as np

from keras import utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import get_custom_objects
from sklearn.metrics import classification_report, confusion_matrix

import openpyxl

#sys.pathに追加（必要なのか調査が必要）
sys.path.append('modules')

# 上にあったのと同じ？ → predictように変更
def _get_indice_pred(feature, maxlen):
    indices = np.zeros((maxlen), dtype=np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(spp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = spp.piece_to_id(token)
        except:
            logging.warn('unknown')
            indices[t] = spp.piece_to_id('<unk>')
            
    return indices, tokens


# SentencePieceProccerモデルの読込
spp = spm.SentencePieceProcessor()
spp.Load('./downloads/bert-wiki-ja/wiki-ja.model')

# BERTの学習したモデルの読込（ダウンロードした？勝手に保存される？）
model_filename = './downloads/models/knbc_finetuning.model'
model = load_model(model_filename, custom_objects=get_custom_objects())
#model = load_model(model_filename, custom_objects=SeqSelfAttention.get_custom_objects())
model = Model(inputs=model.input, outputs=[model.output, model.get_layer('Encoder-12-MultiHeadSelfAttention').output])
# ↑ここでmodel = Model(inputs=a, outputs=b) としてAttentionも出すようにする。


# 上のと同じのを入れると思われるため、消していいかも(ファイルを分けるなら必要)
#SEQ_LEN = 103#206
maxlen = SEQ_LEN

y_train = []
#for i in range(file_count):
for i in range(3):
    n_file = str(i+1).zfill(3)
    file_name = "features_" + n_file + ".csv"
    f_path = ("./datasets/pred_labeling/" + file_name)
    if not os.path.isfile(f_path):
        continue

    df_tests_features = pd.read_csv(f_path)
    print("File Name: ", file_name)
    
    #excelファイル保管用
    excel_file = './attention_excel/attention_' + n_file + '.xlsx'
    writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
        
    wb = openpyxl.Workbook()
    sheet = wb.active
    #sheet.title = 'test_sheet_1'
    wb.save(excel_file)
        
        
    pred_list = []
    good_ratio_list = []
#    for j in range(len(df_tests_features)):
    for j in range(2):
        feature = df_tests_features.loc[j]['feature']

        test_features = []
        indices, tokens = _get_indice_pred(feature, maxlen)
        test_features.append(indices)

        #勝手に追加
        test_features = np.array(test_features)

        test_segments = np.zeros(
            (len(test_features), maxlen), dtype=np.float32)

        # model = Modelを使えば推定　predict[0][0]　２次元のリストで返せる。
        predicted = model.predict([test_features, test_segments])#.argmax(axis=1)
        #predict = model.predict(test_features)

        y_pred = predicted[0].argmax(axis=1)
        print("tokens: ", tokens)
        print("predict: ", y_pred[0])
        
        if y_pred[0] > 0.5:
            pred_list.append([1])
        else:
            pred_list.append([-1])
        
        
        # 高評価度算出
        good = df_tests_features.loc[j]['good']
        bad = df_tests_features.loc[j]['bad']
        
        if bad == 0:
            good_ratio = [0]
        else:
            good_ratio = [good/bad]
        
        good_ratio_list.append(good_ratio)
        
        
        # 入力シーケンスはpad_sequenceにより、以下の様に0でpre paddingしています。
        # [0 0 0 0 x1(300) x2(300) x3(300)] ←３００は (None, 11, 300) の
        # Attention Weightは入力シーケンスに対応して計算されるため、
        # 入力シーケンスのpadding分シフトします。
        #weights = [w.max() for w in predicted[1][0][-len(tokens):]]
        weights = [w.max() for w in predicted[1][0]]#[-len(tokens):]
        df = pd.DataFrame([tokens, weights], index=['token', 'weight']).T

        mean = np.asarray(weights).mean()#np.asarray　参照コピー

        df['rank'] = df['weight'].rank(ascending=False)#ランキング
        # wから平均を引いた値が0より大きいものだけ（偏差）
        df['normalized'] = df['weight'].apply(lambda w: max(w - mean, 0))#行全体や列全体に対して、同じ操作
        df['weight'] = df['weight'].astype('float32')
        df['attention'] = df['normalized'] > 0
        # df.style.background_gradient で色つけ
        df = df.style.background_gradient(cmap='Blues', subset=['normalized'])

        # excel に保存
        sheetname="comment" + str(j)
        with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a") as writer:
            df.to_excel(writer, sheet_name=sheetname, index=False)
        
        #display(df)
        
    y = np.array(pred_list)*np.array(good_ratio_list)
    
    if y.mean() > 0:
        y_train.append(1) 
    else:
        y_train.append(0) 
        
    print("pred_list: ", pred_list)
    print("good_ratio_list: ", good_ratio_list)
    print()

print("y_train: ", y_train)
        

File Name:  features_001.csv
tokens:  ['[CLS]', '▁イギリス', 'から', '戻って', '来た', '▁', '布', '袋', '寅', '泰', '氏は', 'どう', 'する', 'の', 'かね', '▁', '緊急', '事態', '宣言', 'に', '▁', '劇場', 'とか', 'は', '含まれ', 'ない', 'みたい', 'だけ', 'ど', '▁', '全国', 'から', 'ファン', 'を', '▁', '武道', '館', 'に', '来', 'させる', 'の', 'かな', '▁', '中止', 'もしくは', '▁', '武道', '館', 'に', '来', 'なくても', '払い', '戻し', 'します', '▁', '位の', '措置', 'は', '取った', '方が', 'いい', 'と思う', '[SEP]']
predict:  1
tokens:  ['[CLS]', '▁', 'こういう', '事がある', 'から', '鬼', '滅', 'は', '面', '白', 'さ', 'も', '運', 'もあった', 'んだ', 'ろう', 'な', '▁', '鬼', '滅', 'が', '今', 'の', '時期に', 'やって', 'たら', '記録は', 'どう', 'なっていた', 'か', '[SEP]']
predict:  1
pred_list:  [[1], [1]]
good_ratio_list:  [[7.961538461538462], [11.466666666666667]]

File Name:  features_003.csv
tokens:  ['[CLS]', '▁', '自社', 'ビル', 'を', '所有する', 'の', 'は', '、', '▁', '会社', 'として', '、', '素晴らしい', '事', 'です', 'が', '、', '▁', '経営', 'が', '傾', 'く', '一つの', '原因', 'で', '、', '▁', '自社', 'ビル', 'の建設', '費用', 'が', '重', '荷', 'になり', '、', '▁', 'という', 'の', 'をよく', '見かけ

In [44]:
pred_list=[[1], [1]]
good_ratio_list=[[6.413897280966768], [9.019292604501608]]

y = np.array(pred_list)*np.array(good_ratio_list)

print(y)
print(y.mean())

[[6.41389728]
 [9.0192926 ]]
7.716594942734188


## 推論時にAttention Weightを可視化する

In [45]:
tests_features_df = pd.read_csv('./datasets/pred_labeling/features_001.csv')

In [50]:
from text_vectorian import SentencePieceVectorian
vectorian = SentencePieceVectorian()

/tmp/.keras/.models/wikija-sentencepiece_300.model
/tmp/.keras/.models/wikija-sentencepieced_word2vec_300.model
/tmp/.keras/.models/wikija-sentencepieced_word2vec_300.model.wv.vectors.npy
/tmp/.keras/.models/wikija-sentencepieced_word2vec_300.model.trainables.syn1neg.npy


In [51]:
# 保存
#export_dir='./saved_model'
#tf.saved_model.save(bert_classifier, export_dir=export_dir)

In [52]:
# 復元
#reloaded = tf.saved_model.load(export_dir)
#reloaded_result = reloaded([my_examples['input_word_ids'],
#                            my_examples['input_mask'],
#                            my_examples['input_type_ids']], training=False)
#
#original_result = bert_classifier(my_examples, training=False)
#
# The results are (nearly) identical:
#print(original_result.numpy())
#print()
#print(reloaded_result.numpy())